# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/chrisdaems/Documents/dend/Data Modeling Cassandra/dend_project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)



#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
# Create keyspace on a single node evaluation cluster
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class':'SimpleStrategy','replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# CREATE Tables that supports the queries above
# Table 1 - song_session, a table supporting data about the song and session of sparkify
query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + "(sessionId int, itemInSession int, song text, length float, artist text, "
query = query + " PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Table 2 - user_session, a table supporting data about the user and session of sparkify
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, "
query = query + " PRIMARY KEY((userId, sessionId),itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

# Table 3- user_history_song, a table that holds history info about users listening to a song 
query = "CREATE TABLE IF NOT EXISTS user_history_song "
query = query + "(userId int, artist text, song text, firstName text, lastName text, "
query = query + " PRIMARY KEY((song), userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)


#### indexes of each column in the CSV

- artist[0]

- firstName[1]

- gender[2]

- itemInSession[3]

- lastName[4]

- length[5]

- level[6]

- location[7]

- sessionId[8]

- song[9]

- userId[10]



In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query_song_session = "INSERT INTO song_session (sessionId, itemInSession, song, length, artist)"
        query_song_session = query_song_session + "VALUES (%s,%s,%s,%s,%s) "
        
        query_user_session = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query_user_session = query_user_session + "VALUES (%s,%s,%s,%s,%s,%s,%s) "
        
        query_user_history_song = "INSERT INTO user_history_song (song, userId, firstName, lastName)"
        query_user_history_song = query_user_history_song + "VALUES (%s,%s,%s,%s) "
      
        try:
            #                                    sessionId   , itemInSession , song   , length        , artist 
            session.execute(query_song_session, (int(line[8]), int(line[3])  , line[9], float(line[5]), line[0]))
            
            #                                    userId       , sessionId   , itemInSession, artist , song   , firstName, lastName 
            session.execute(query_user_session, (int(line[10]), int(line[8]), int(line[3]) , line[0], line[9], line[1]  , line[4]))
            
            #                                         song   , userId       , firstName , lastName 
            session.execute(query_user_history_song, (line[9], int(line[10]), line[1]   , line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "select artist, song, length FROM song_session WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('artist:', row.artist, ' - song:', row.song, ' - length:', row.length)

artist: Faithless  - song: Music Matters (Mark Knight Dub)  - length: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "select * FROM user_session WHERE userId=10 and sessionId=182 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('artist:', row.artist, ' - song:', row.song, ' - user name:', row.firstname, row.lastname)

                    

artist: Down To The Bone  - song: Keep On Keepin' On  - user name: Sylvie Cruz
artist: Three Drives  - song: Greece 2000  - user name: Sylvie Cruz
artist: Sebastien Tellier  - song: Kilometer  - user name: Sylvie Cruz
artist: Lonnie Gordon  - song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  - user name: Sylvie Cruz


In [12]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own
query = "select * FROM user_history_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('user name:', row.firstname, row.lastname)
                    

user name: Jacqueline Lynch
user name: Tegan Levine
user name: Sara Johnson


### Drop the tables before closing out the sessions

In [13]:
## TO-DO: Drop the table before closing out the sessions
session.execute('DROP TABLE IF EXISTS song_session')
session.execute('DROP TABLE IF EXISTS user_session')
session.execute('DROP TABLE IF EXISTS user_history_song')


### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()